In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpt
import warnings
warnings.simplefilter('ignore')

In [2]:
from astropy import units as u
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from astropy.io import fits

In [3]:
def get_center_in_ait(coord):
    #create wcs
    #resolution 1"/pixel
    cdelt = 1./60./60.
    nx = np.round(360./cdelt)
    ny = np.round(180./cdelt)
    w = WCS(naxis=2)
    w.wcs.crpix = [float(nx/2.), float(ny/2.)]
    w.wcs.cdelt = np.array([-cdelt,cdelt])
    w.wcs.crval = [0., 0.]
    w.wcs.ctype = ["GLON-AIT", "GLAT-AIT"]
    w.array_shape = [nx,ny]
    px, py = w.wcs_world2pix(coord.l,coord.b, 1)
    cx,cy = np.mean(px),np.mean(py)
    c = w.wcs_pix2world(cx,cy, 1)
    
    return c

In [4]:
def get_center_in_car(coord,cent):
    #create wcs
    #resolution 1"/pixel
    cdelt = 1./60./60./60.
    nx = np.round(360./cdelt)
    ny = np.round(180./cdelt)
    w = WCS(naxis=2)
    w.wcs.crpix = [float(nx/2.), float(ny/2.)]
    w.wcs.cdelt = np.array([-cdelt,cdelt])
    w.wcs.crval = [cent[0], cent[1]]
    w.wcs.ctype = ["GLON-CAR", "GLAT-CAR"]
    w.array_shape = [nx,ny]
    px, py = w.wcs_world2pix(coord.l,coord.b, 1)
    cx,cy = np.mean(px),np.mean(py)
    c = w.wcs_pix2world(cx,cy, 1)
    
    return c

def get_centroid(coord,frame="galactic",equinox="J2000",unit=u.deg,max_itr=5):
    cent = get_center_in_ait(wcoord)
    for i in range(max_itr):
        cent = get_center_in_car(wcoord,cent)
        
    cent = SkyCoord(cent[0],cent[1],unit=unit,frame=frame,equinox=equinox)
    return cent

def ang_sep(coord,cent):
    sep_data = {"lon":[],"lat":[],"sep":[]}
    for i in range(len(coord)):
        sep_data["lon"].append(coord.l[i])
        sep_data["lat"].append(coord.b[i])
        sep_data["sep"].append(coord[i].separation(cent))
         
    sep_data["lon"] *=u.deg
    sep_data["lat"] *=u.deg
    sep_data["sep"] *=u.deg
    return pd.DataFrame.from_dict(sep_data)
        

In [5]:
lon=np.array([248,235.2,240,257.1,237.4,266.2,261.3])
lat=np.array([-49.2,-49.1,-48.6,-48.4,-48.2,-44.4,-40.2])
unit=u.deg
equinox="J2000"
frame="galactic"
wcoord = SkyCoord(lon,lat,unit=unit,frame=frame,equinox=equinox)
cent = get_centroid(wcoord,frame=frame,equinox=equinox,unit=unit,max_itr=10)
cent

<SkyCoord (Galactic): (l, b) in deg
    (249.783022, -47.4539962)>

In [6]:
ang_sep(wcoord,cent)

,lon,lat,sep
0,248.0,-49.2,2.110294
1,235.2,-49.1,9.828033
2,240.0,-48.6,6.637463
3,257.1,-48.4,4.991308
4,237.4,-48.2,8.337871
5,266.2,-44.4,11.795524
6,261.3,-40.2,11.008933


In [7]:
lon=np.array([314.7,314.5,309.5,304.3])
lat=np.array([32.8,35.3,38.3,40.7])
unit=u.deg
equinox="J2000"
frame="galactic"
wcoord = SkyCoord(lon,lat,unit=unit,frame=frame,equinox=equinox)
cent = get_centroid(wcoord,frame=frame,equinox=equinox,unit=unit,max_itr=10)
cent

<SkyCoord (Galactic): (l, b) in deg
    (310.90915972, 36.85002937)>

In [8]:
ang_sep(wcoord,cent)

,lon,lat,sep
0,314.7,32.8,5.106429
1,314.5,35.3,3.289918
2,309.5,38.3,1.830173
3,304.3,40.7,6.428873


In [9]:
lon=np.array([284.3,285.5,250.1,283.3,251.2])
lat=np.array([76.4,75,76.3,71.6,73.3])
unit=u.deg
equinox="J2000"
frame="galactic"
wcoord = SkyCoord(lon,lat,unit=unit,frame=frame,equinox=equinox)
cent = get_centroid(wcoord,frame=frame,equinox=equinox,unit=unit,max_itr=10)
print(cent)
ang_sep(wcoord,cent)

<SkyCoord (Galactic): (l, b) in deg
    (271.18906616, 75.12232267)>


,lon,lat,sep
0,284.3,76.4,3.459616
1,285.5,75.0,3.682237
2,250.1,76.3,5.305474
3,283.3,71.6,4.925528
4,251.2,73.3,5.703509
